1. STCE: Transformer + CNN
Architecture: Dense→n×(Multi‑Head Attention + residual) → n×(Conv1D + residual) → Dense output

Window: 7 days (6→1)

Metrics: RMSE, MAE

In [ ]:
# ─── Imports & Seeds ───────────────────────────────────────────────────────────
import os, glob, random, time
import numpy as np, pandas as pd
import tensorflow as tf
from math import sqrt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import (
    Dense, Dropout, LayerNormalization,
    MultiHeadAttention, Conv1D, Add
)
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.optimizers import Adam

# reproducibility
seed = 42
np.random.seed(seed)
random.seed(seed)
tf.random.set_seed(seed)

# ─── PARAMETERS ────────────────────────────────────────────────────────────────
INPUT_DIR   = "data/raw"
OUTPUT_DIR  = "outputs"

# ensure directories exist
os.makedirs(INPUT_DIR,  exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

# window & split sizes
WINDOW_SIZE = 7
SEQ_LENGTH  = WINDOW_SIZE - 1

# train‑set size per crop
TRAIN_SIZES = {
    "Cucumber": 2826,
    "LongBean": 2826,
    "Tomato":   2826
}

# model hyperparameters
D_MODEL       = 64
N_HEADS       = 4
CONV_FILTERS  = 64
CONV_KERNEL   = 4
DROPOUT_RATE  = 0.2
N_LAYERS      = 2

# training
EPOCHS     = 100
BATCH_SIZE = 32


# ─── MODEL DEFINITION ──────────────────────────────────────────────────────────
def transformer_model_with_cnn(
    seq_len, num_feats, d_model, n_heads,
    conv_filters, conv_kernel, dropout_rate, n_layers
):
    inp = Input(shape=(seq_len, num_feats))
    x = inp

    # attention + residual
    for _ in range(n_layers):
        attn = MultiHeadAttention(num_heads=n_heads, key_dim=d_model)(x, x)
        attn = Dropout(dropout_rate)(attn)
        attn = LayerNormalization(epsilon=1e-6)(attn)
        x = Add()([x, attn])

    # convolution + residual
    for _ in range(n_layers):
        c = Conv1D(filters=conv_filters, kernel_size=conv_kernel,
                   activation="relu", padding="same")(x)
        c = Dropout(dropout_rate)(c)
        c = LayerNormalization(epsilon=1e-6)(c)
        x = Add()([x, c])

    out = Dense(units=num_feats)(x)
    return Model(inputs=inp, outputs=out)


# ─── SHEET PROCESSOR ───────────────────────────────────────────────────────────
def process_sheet(df, train_size):
    # parse & index by date
    df["Tarikh"] = pd.to_datetime(df["Tarikh"]).dt.strftime("%d/%m/%Y")
    df.index = df["Tarikh"]
    df = df.drop("Tarikh", axis=1).apply(pd.to_numeric, errors="coerce")

    # target series
    series = df["WholesalePriceNew"].values.reshape(-1, 1)
    train, test = series[:train_size], series[train_size:]

    # scale 0–1
    scaler = MinMaxScaler((0, 1))
    train_s = scaler.fit_transform(train)
    test_s  = scaler.transform(test)

    # sliding windows
    def slide(arr, w): return np.array([arr[i:i+w] for i in range(len(arr)-w+1)])
    w = WINDOW_SIZE
    tx = slide(train_s, w); ty = tx[:, -1]; tx = tx[:, :-1]
    vx = slide(test_s, w);  vy = vx[:, -1]; vx = vx[:, :-1]

    # build & train
    model = transformer_model_with_cnn(
        seq_len=w-1,
        num_feats=1,
        d_model=D_MODEL,
        n_heads=N_HEADS,
        conv_filters=CONV_FILTERS,
        conv_kernel=CONV_KERNEL,
        dropout_rate=DROPOUT_RATE,
        n_layers=N_LAYERS
    )
    model.compile(loss=MeanSquaredError(), optimizer=Adam(1e-3))
    model.fit(tx, ty, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=2)

    # predict & invert scale
    preds = model.predict(vx)[:, -1, :]
    preds_r = scaler.inverse_transform(preds)
    vy_r    = scaler.inverse_transform(vy)

    rmse = sqrt(mean_squared_error(vy_r, preds_r))
    mae  = mean_absolute_error(vy_r, preds_r)
    return vy_r, preds_r, rmse, mae


# ─── MAIN LOOP ─────────────────────────────────────────────────────────────────
for fp in glob.glob(os.path.join(INPUT_DIR, "*_FillKNN.xlsx")):
    crop = os.path.basename(fp).split("_")[0]
    ts   = TRAIN_SIZES.get(crop)
    out_file = f"{crop}_STCE_results.xlsx"

    # load & prepare writer
    xls    = pd.ExcelFile(fp)
    writer = pd.ExcelWriter(os.path.join(OUTPUT_DIR, out_file),
                            engine="xlsxwriter")

    print(f"\n➡️ Processing {crop}…")
    for sheet in xls.sheet_names:
        df = pd.read_excel(fp, sheet_name=sheet)
        actual, pred, rmse, mae = process_sheet(df, ts)

        # save sheet
        out_df = pd.DataFrame({
            "Actual":    actual.flatten(),
            "Predicted": pred.flatten(),
            "RMSE":      rmse,
            "MAE":       mae
        })
        out_df.to_excel(writer, sheet_name=sheet, index=False)
        print(f"   • {sheet}: RMSE={rmse:.3f}, MAE={mae:.3f}")

    writer.close()
    print(f"✅ Saved: {os.path.join(OUTPUT_DIR, out_file)}")



➡️ Processing Cucumber…
Epoch 1/100
89/89 - 6s - 67ms/step - loss: 0.5938
Epoch 2/100
89/89 - 1s - 8ms/step - loss: 0.1184
Epoch 3/100
89/89 - 1s - 8ms/step - loss: 0.0585
Epoch 4/100
89/89 - 1s - 8ms/step - loss: 0.0363
Epoch 5/100
89/89 - 1s - 8ms/step - loss: 0.0269
Epoch 6/100
89/89 - 1s - 8ms/step - loss: 0.0202
Epoch 7/100
89/89 - 1s - 8ms/step - loss: 0.0160
Epoch 8/100
89/89 - 1s - 8ms/step - loss: 0.0136
Epoch 9/100
89/89 - 1s - 8ms/step - loss: 0.0121
Epoch 10/100
89/89 - 1s - 9ms/step - loss: 0.0103
Epoch 11/100
89/89 - 1s - 10ms/step - loss: 0.0092
Epoch 12/100
89/89 - 1s - 9ms/step - loss: 0.0081
Epoch 13/100
89/89 - 1s - 8ms/step - loss: 0.0074
Epoch 14/100
89/89 - 1s - 8ms/step - loss: 0.0067
Epoch 15/100
89/89 - 1s - 8ms/step - loss: 0.0065
Epoch 16/100
89/89 - 1s - 8ms/step - loss: 0.0059
Epoch 17/100
89/89 - 1s - 9ms/step - loss: 0.0055
Epoch 18/100
89/89 - 1s - 9ms/step - loss: 0.0053
Epoch 19/100
89/89 - 1s - 9ms/step - loss: 0.0051
Epoch 20/100
89/89 - 1s - 10ms/s

2. TCE: Transformer + CNN + Positional Encoding + MLP
Positional Encoding: sinusoidal

Architecture: Dense→PosEnc→Dropout → n×(Attention + residual) → n×(Conv1D + residual) → MLP(128→64) → Dense output

Metrics: RMSE, MAE, MAPE

In [ ]:
# ─── Imports & Seeds ───────────────────────────────────────────────────────────
import os, glob, random, time
import numpy as np, pandas as pd
import tensorflow as tf
from math import sqrt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import (Dense, Dropout, LayerNormalization,
                                     MultiHeadAttention, Conv1D, Add, Layer)
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.optimizers import Adam

# reproducibility
seed = 42
np.random.seed(seed)
random.seed(seed)
tf.random.set_seed(seed)

# ─── PARAMETERS ────────────────────────────────────────────────────────────────
INPUT_DIR   = "data/raw"
OUTPUT_DIR  = "outputs"

# ensure directories exist
os.makedirs(INPUT_DIR,  exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

# train set size per crop (adjust as needed)
TRAIN_SIZES = {
    "Cucumber": 2826,
    "LongBean": 2826,
    "Tomato":   2826,
    "KubisBulat": 2856
}

# sliding‑window
WINDOW_SIZE = 7
SEQ_LENGTH  = WINDOW_SIZE - 1

# model hyperparameters
D_MODEL       = 64
N_HEADS       = 4
CONV_FILTERS  = 64
CONV_KERNEL   = 12
DROPOUT_RATE  = 0.2
N_LAYERS      = 2
MLP_UNITS_1   = 128
MLP_UNITS_2   = 64

# training params
EPOCHS     = 100
BATCH_SIZE = 32


# ─── Positional Encoding Layer ────────────────────────────────────────────────
class PositionalEncoding(Layer):
    def __init__(self, d_model, **kwargs):
        super(PositionalEncoding, self).__init__(**kwargs)
        self.d_model = d_model

    def get_config(self):
        cfg = super(PositionalEncoding, self).get_config()
        cfg.update({"d_model": self.d_model})
        return cfg

    def call(self, x):
        pos = tf.range(tf.shape(x)[1], dtype=tf.float32)[:, tf.newaxis]
        div = tf.exp(tf.range(0, self.d_model, 2, dtype=tf.float32) *
                     (-tf.math.log(10000.0) / self.d_model))
        angle = pos * div
        pe = tf.concat([tf.sin(angle), tf.cos(angle)], axis=-1)[tf.newaxis, ...]
        return x + pe


# ─── Model Definition ──────────────────────────────────────────────────────────
def build_tce_model(seq_len, num_feats):
    inp = Input(shape=(seq_len, num_feats))
    # project to d_model dims
    x = Dense(D_MODEL)(inp)
    # add positional encoding
    x = PositionalEncoding(D_MODEL)(x)
    x = Dropout(DROPOUT_RATE)(x)

    # self‑attention blocks
    for _ in range(N_LAYERS):
        attn = MultiHeadAttention(num_heads=N_HEADS, key_dim=D_MODEL)(x, x)
        attn = Dropout(DROPOUT_RATE)(attn)
        attn = LayerNormalization(epsilon=1e-6)(attn)
        x = Add()([x, attn])

    # convolutional residual blocks
    for _ in range(N_LAYERS):
        c = Conv1D(filters=CONV_FILTERS, kernel_size=CONV_KERNEL,
                   activation="relu", padding="same")(x)
        c = Dropout(DROPOUT_RATE)(c)
        c = LayerNormalization(epsilon=1e-6)(c)
        # ensure dims match
        if c.shape[-1] != x.shape[-1]:
            x = Dense(c.shape[-1])(x)
        x = Add()([x, c])

    # MLP head
    x = Dense(MLP_UNITS_1, activation="relu")(x)
    x = Dropout(DROPOUT_RATE)(x)
    x = Dense(MLP_UNITS_2, activation="relu")(x)
    x = Dropout(DROPOUT_RATE)(x)

    out = Dense(num_feats)(x)
    return Model(inputs=inp, outputs=out)


# ─── Sheet Processor ───────────────────────────────────────────────────────────
def process_sheet(df, train_size):
    # format and index
    df["Tarikh"] = pd.to_datetime(df["Tarikh"]).dt.strftime("%d/%m/%Y")
    df.index = df["Tarikh"]
    df = df.drop("Tarikh", axis=1).apply(pd.to_numeric, errors="coerce")

    series = df["WholesalePriceNew"].values.reshape(-1, 1)
    train, test = series[:train_size], series[train_size:]

    # scale 0–1
    scaler = MinMaxScaler((0, 1))
    train_s = scaler.fit_transform(train)
    test_s  = scaler.transform(test)

    # sliding-window
    def slide(arr, w): return np.array([arr[i:i+w] for i in range(len(arr)-w+1)])
    w = WINDOW_SIZE
    tx = slide(train_s, w); ty = tx[:, -1]; tx = tx[:, :-1]
    vx = slide(test_s, w);  vy = vx[:, -1]; vx = vx[:, :-1]

    # build & train
    model = build_tce_model(seq_len=w-1, num_feats=1)
    model.compile(loss=MeanSquaredError(), optimizer=Adam(1e-3))
    model.fit(tx, ty, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=2)

    # predict & invert scaling
    preds = model.predict(vx)[:, -1, :]
    preds_r = scaler.inverse_transform(preds)
    vy_r    = scaler.inverse_transform(vy)

    # metrics
    rmse = sqrt(mean_squared_error(vy_r, preds_r))
    mae  = mean_absolute_error(vy_r, preds_r)
    mape = mean_absolute_percentage_error(vy_r, preds_r) * 100
    return vy_r, preds_r, rmse, mae, mape


# ─── Main Loop ─────────────────────────────────────────────────────────────────
for fp in glob.glob(os.path.join(INPUT_DIR, "*_FillKNN.xlsx")):
    crop      = os.path.basename(fp).split("_")[0]
    train_sz  = TRAIN_SIZES.get(crop, WINDOW_SIZE*10)  # fallback if missing
    out_fname = f"{crop}_TCE_results.xlsx"

    xls    = pd.ExcelFile(fp)
    writer = pd.ExcelWriter(os.path.join(OUTPUT_DIR, out_fname),
                            engine="xlsxwriter")

    print(f"\n➡️ Running TCE on {crop}…")
    for sheet in xls.sheet_names:
        df = pd.read_excel(fp, sheet_name=sheet)
        actual, pred, rmse, mae, mape = process_sheet(df, train_sz)

        results = pd.DataFrame({
            "Actual":    actual.flatten(),
            "Predicted": pred.flatten(),
            "RMSE":      rmse,
            "MAE":       mae,
            "MAPE":      mape
        })
        results.to_excel(writer, sheet_name=sheet, index=False)
        print(f"   • {sheet}: RMSE={rmse:.3f}, MAE={mae:.3f}, MAPE={mape:.2f}%")

    writer.close()
    print(f"✅ Saved → {os.path.join(OUTPUT_DIR, out_fname)}")


3. STLE: Transformer + LSTM
Architecture: n×(Multi‑Head Attention + residual) → n×(LSTM + residual) → Dense output

Metrics: RMSE, MAE

In [ ]:
# ─── Imports & Seeds ───────────────────────────────────────────────────────────
import os, glob, random, time
import numpy as np, pandas as pd
import tensorflow as tf
from math import sqrt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import (
    Dense, Dropout, LayerNormalization,
    MultiHeadAttention, LSTM, Add
)
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.optimizers import Adam

# reproducibility
seed = 42
np.random.seed(seed)
random.seed(seed)
tf.random.set_seed(seed)

# ─── PARAMETERS ────────────────────────────────────────────────────────────────
INPUT_DIR   = "data/raw"
OUTPUT_DIR  = "outputs"

# ensure directories exist
os.makedirs(INPUT_DIR,  exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

# train‑set size per crop
TRAIN_SIZES = {
    "Cucumber":   2826,
    "LongBean":   2826,
    "KubisBulat": 2856,
    "Tomato":     2886
}

# sliding‑window
WINDOW_SIZE = 7
SEQ_LENGTH  = WINDOW_SIZE - 1

# model hyperparameters
D_MODEL      = 64
N_HEADS      = 4
LSTM_UNITS   = 64
DROPOUT_RATE = 0.2
N_LAYERS     = 2

# training
EPOCHS     = 100
BATCH_SIZE = 32


# ─── MODEL DEFINITION ──────────────────────────────────────────────────────────
def transformer_model_with_lstm(seq_len, num_feats, d_model, n_heads, lstm_units, dropout_rate, n_layers):
    inp = Input(shape=(seq_len, num_feats))
    x   = inp

    # self‑attention + residual
    for _ in range(n_layers):
        attn = MultiHeadAttention(num_heads=n_heads, key_dim=d_model)(x, x)
        attn = Dropout(dropout_rate)(attn)
        attn = LayerNormalization(epsilon=1e-6)(attn)
        x    = Add()([x, attn])

    # LSTM + residual
    for _ in range(n_layers):
        lstm = LSTM(units=lstm_units, return_sequences=True)(x)
        lstm = Dropout(dropout_rate)(lstm)
        lstm = LayerNormalization(epsilon=1e-6)(lstm)
        x    = Add()([x, lstm])

    out = Dense(units=num_feats)(x)
    return Model(inputs=inp, outputs=out)


# ─── SHEET PROCESSOR ───────────────────────────────────────────────────────────
def process_sheet(df, train_size):
    # format & index
    df["Tarikh"] = pd.to_datetime(df["Tarikh"]).dt.strftime("%d/%m/%Y")
    df.index     = df["Tarikh"]
    df = df.drop("Tarikh", axis=1).apply(pd.to_numeric, errors="coerce")

    # target series
    series = df["WholesalePriceNew"].values.reshape(-1, 1)
    train, test = series[:train_size], series[train_size:]

    # scale to [0,1]
    scaler = MinMaxScaler((0, 1))
    train_s = scaler.fit_transform(train)
    test_s  = scaler.transform(test)

    # sliding windows
    def slide(arr, w):
        return np.array([arr[i : i + w] for i in range(len(arr) - w + 1)])
    w  = WINDOW_SIZE
    tx = slide(train_s, w); ty = tx[:, -1]; tx = tx[:, :-1]
    vx = slide(test_s,  w); vy = vx[:, -1]; vx = vx[:, :-1]

    # build & train
    model = transformer_model_with_lstm(
        seq_len      = w - 1,
        num_feats    = 1,
        d_model      = D_MODEL,
        n_heads      = N_HEADS,
        lstm_units   = LSTM_UNITS,
        dropout_rate = DROPOUT_RATE,
        n_layers     = N_LAYERS
    )
    model.compile(loss=MeanSquaredError(), optimizer=Adam(1e-3))
    model.fit(tx, ty, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=2)

    # predict & invert scale
    preds   = model.predict(vx)[:, -1, :]
    preds_r = scaler.inverse_transform(preds)
    vy_r    = scaler.inverse_transform(vy)

    # compute metrics
    rmse = sqrt(mean_squared_error(vy_r, preds_r))
    mae  = mean_absolute_error(vy_r, preds_r)
    return vy_r, preds_r, rmse, mae


# ─── MAIN LOOP ─────────────────────────────────────────────────────────────────
for fp in glob.glob(os.path.join(INPUT_DIR, "*_FillKNN.xlsx")):
    crop     = os.path.basename(fp).split("_")[0]
    ts       = TRAIN_SIZES.get(crop, WINDOW_SIZE*10)
    out_file = f"{crop}_STLE_results.xlsx"

    xls    = pd.ExcelFile(fp)
    writer = pd.ExcelWriter(os.path.join(OUTPUT_DIR, out_file), engine="xlsxwriter")

    print(f"\n➡️ Processing {crop} with STLE…")
    for sheet in xls.sheet_names:
        df     = pd.read_excel(fp, sheet_name=sheet)
        actual, pred, rmse, mae = process_sheet(df, ts)

        results = pd.DataFrame({
            "Actual":    actual.flatten(),
            "Predicted": pred.flatten(),
            "RMSE":      rmse,
            "MAE":       mae
        })
        results.to_excel(writer, sheet_name=sheet, index=False)
        print(f"   • {sheet}: RMSE={rmse:.3f}, MAE={mae:.3f}")

    writer.close()
    print(f"✅ Saved → {os.path.join(OUTPUT_DIR, out_file)}")


4. TFE: Transformer + FFN
Flow: n×(Attention + residual) → n×(Position‑wise Feed‑Forward + residual) → Dense

Metrics: RMSE, MAE

In [ ]:
# ─── Imports & Seeds ───────────────────────────────────────────────────────────
import os, glob, random, time
import numpy as np, pandas as pd
import tensorflow as tf
from math import sqrt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense, Dropout, LayerNormalization, MultiHeadAttention, Add
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.optimizers import Adam

# reproducibility
seed = 42
np.random.seed(seed)
random.seed(seed)
tf.random.set_seed(seed)

# ─── PARAMETERS ────────────────────────────────────────────────────────────────
INPUT_DIR   = "data/raw"
OUTPUT_DIR  = "outputs"

# ensure directories exist
os.makedirs(INPUT_DIR,  exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

# train‑set size per crop
TRAIN_SIZES = {
    "Cucumber":   2826,
    "LongBean":   2826,
    "KubisBulat": 2856,
    "Tomato":     2886
}

# sliding‑window
WINDOW_SIZE = 7
SEQ_LENGTH  = WINDOW_SIZE - 1

# model hyperparameters
D_MODEL      = 64
N_HEADS      = 4
DENSE_UNITS  = 128
DROPOUT_RATE = 0.2
N_LAYERS     = 2

# training
EPOCHS     = 100
BATCH_SIZE = 32


# ─── MODEL DEFINITION ──────────────────────────────────────────────────────────
def build_tfe_model(seq_len, num_feats):
    inp = Input(shape=(seq_len, num_feats))
    x   = inp

    # Multi‑Head Self‑Attention + residual
    for _ in range(N_LAYERS):
        attn = MultiHeadAttention(num_heads=N_HEADS, key_dim=D_MODEL)(x, x)
        attn = Dropout(DROPOUT_RATE)(attn)
        attn = LayerNormalization(epsilon=1e-6)(attn)
        x    = Add()([x, attn])

    # Position‑wise Feed‑Forward + residual
    for _ in range(N_LAYERS):
        ff = Dense(units=DENSE_UNITS, activation="relu")(x)
        ff = Dropout(DROPOUT_RATE)(ff)
        ff = LayerNormalization(epsilon=1e-6)(ff)
        x  = Add()([x, ff])

    out = Dense(units=num_feats)(x)
    return Model(inputs=inp, outputs=out)


# ─── SHEET PROCESSOR ───────────────────────────────────────────────────────────
def process_sheet(df, train_size):
    # format dates & index
    df["Tarikh"] = pd.to_datetime(df["Tarikh"]).dt.strftime("%d/%m/%Y")
    df.index     = df["Tarikh"]
    df = df.drop("Tarikh", axis=1).apply(pd.to_numeric, errors="coerce")

    # extract series
    series = df["WholesalePriceNew"].values.reshape(-1, 1)
    train, test = series[:train_size], series[train_size:]

    # scale to [0,1]
    scaler = MinMaxScaler((0, 1))
    train_s = scaler.fit_transform(train)
    test_s  = scaler.transform(test)

    # sliding windows
    def slide(arr, w): return np.array([arr[i:i+w] for i in range(len(arr)-w+1)])
    w  = WINDOW_SIZE
    tx = slide(train_s, w); ty = tx[:, -1]; tx = tx[:, :-1]
    vx = slide(test_s,  w); vy = vx[:, -1]; vx = vx[:, :-1]

    # build & train
    model = build_tfe_model(seq_len=w-1, num_feats=1)
    model.compile(loss=MeanSquaredError(), optimizer=Adam(1e-3))
    model.fit(tx, ty, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=2)

    # predict & invert scaling
    preds   = model.predict(vx)[:, -1, :]
    preds_r = scaler.inverse_transform(preds)
    vy_r    = scaler.inverse_transform(vy)

    # metrics
    rmse = sqrt(mean_squared_error(vy_r, preds_r))
    mae  = mean_absolute_error(vy_r, preds_r)
    return vy_r, preds_r, rmse, mae


# ─── MAIN LOOP ─────────────────────────────────────────────────────────────────
for filepath in glob.glob(os.path.join(INPUT_DIR, "*_FillKNN.xlsx")):
    crop      = os.path.basename(filepath).split("_")[0]
    train_sz  = TRAIN_SIZES.get(crop, WINDOW_SIZE*10)  # fallback
    out_fname = f"{crop}_TFE_results.xlsx"

    xls    = pd.ExcelFile(filepath)
    writer = pd.ExcelWriter(os.path.join(OUTPUT_DIR, out_fname),
                            engine="xlsxwriter")

    print(f"\n➡️ Processing {crop} with TFE…")
    for sheet in xls.sheet_names:
        df      = pd.read_excel(filepath, sheet_name=sheet)
        actual, pred, rmse, mae = process_sheet(df, train_sz)

        results = pd.DataFrame({
            "Actual":    actual.flatten(),
            "Predicted": pred.flatten(),
            "RMSE":      rmse,
            "MAE":       mae
        })
        results.to_excel(writer, sheet_name=sheet, index=False)
        print(f"   • {sheet}: RMSE={rmse:.3f}, MAE={mae:.3f}")

    writer.close()
    print(f"✅ Saved → {os.path.join(OUTPUT_DIR, out_fname)}")


5. LSTM use as the Comparison

In [ ]:
# ─── Imports & Seeds ───────────────────────────────────────────────────────────
import os, glob, random, time
import numpy as np, pandas as pd
import tensorflow as tf
from math import sqrt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam

# reproducibility
seed = 42
np.random.seed(seed)
random.seed(seed)
tf.random.set_seed(seed)

# ─── PARAMETERS ────────────────────────────────────────────────────────────────
INPUT_DIR    = "data/raw"
OUTPUT_DIR   = "outputs"
TIMESTEPS    = 7     # sliding window length
DAY_PREDICT  = 90    # how many days at end used for testing
EPOCHS       = 100
BATCH_SIZE   = 32

# ensure directories exist
os.makedirs(INPUT_DIR,  exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)


# ─── SHEET PROCESSOR ───────────────────────────────────────────────────────────
def process_sheet(df):
    # format dates & index
    df["Tarikh"] = pd.to_datetime(df["Tarikh"]).dt.strftime("%d/%m/%Y")
    df.index     = df["Tarikh"]
    df = df.drop("Tarikh", axis=1).apply(pd.to_numeric, errors="coerce")

    # extract series and reshape
    series = df["WholesalePriceNew"].values.reshape(-1, 1)

    # train/test split
    train_len = len(series) - DAY_PREDICT
    train, test = series[:train_len], series[train_len:]

    # scale to [0,1]
    scaler = MinMaxScaler((0, 1))
    train_s = scaler.fit_transform(train)
    test_s  = scaler.transform(test)

    # create sliding windows
    x_train, y_train = [], []
    for i in range(TIMESTEPS, len(train_s)):
        x_train.append(train_s[i-TIMESTEPS:i, 0])
        y_train.append(train_s[i, 0])
    x_train = np.array(x_train).reshape(-1, TIMESTEPS, 1)
    y_train = np.array(y_train)

    # build LSTM model
    model = Sequential([
        LSTM(100, return_sequences=True, input_shape=(TIMESTEPS, 1)),
        LSTM(100, return_sequences=False),
        Dense(50, activation="relu"),
        Dense(1)
    ])
    model.compile(optimizer=Adam(1e-3), loss="mse")
    model.fit(x_train, y_train, epochs=EPOCHS,
              batch_size=BATCH_SIZE, verbose=2)

    # prepare test windows
    x_test = []
    full = np.vstack((train_s[-TIMESTEPS:], test_s))
    for i in range(TIMESTEPS, len(full)):
        x_test.append(full[i-TIMESTEPS:i, 0])
    x_test = np.array(x_test).reshape(-1, TIMESTEPS, 1)

    # predict & invert scale
    preds = model.predict(x_test)
    preds_r = scaler.inverse_transform(preds)
    y_test  = test.reshape(-1, 1)

    # metrics
    rmse = sqrt(mean_squared_error(y_test, preds_r))
    mae  = mean_absolute_error(y_test, preds_r)
    mape = mean_absolute_percentage_error(y_test, preds_r) * 100

    return y_test.flatten(), preds_r.flatten(), rmse, mae, mape


# ─── MAIN LOOP ─────────────────────────────────────────────────────────────────
for fp in glob.glob(os.path.join(INPUT_DIR, "*_FillKNN.xlsx")):
    crop      = os.path.basename(fp).split("_")[0]
    out_fname = f"{crop}_LSTM_results.xlsx"
    writer    = pd.ExcelWriter(os.path.join(OUTPUT_DIR, out_fname),
                               engine="xlsxwriter")

    print(f"\n➡️ Running LSTM for {crop}…")
    xls = pd.ExcelFile(fp)
    for sheet in xls.sheet_names:
        df = pd.read_excel(fp, sheet_name=sheet)
        actual, preds, rmse, mae, mape = process_sheet(df)

        # assemble results
        out_df = pd.DataFrame({
            "Actual":    actual,
            "Predicted": preds,
            "RMSE":      rmse,
            "MAE":       mae,
            "MAPE":      mape
        })
        out_df.to_excel(writer, sheet_name=sheet, index=False)
        print(f"   • {sheet}: RMSE={rmse:.3f}, MAE={mae:.3f}, MAPE={mape:.2f}%")

    writer.close()
    print(f"✅ Saved → {os.path.join(OUTPUT_DIR, out_fname)}")


In [ ]:
# ─── Imports & Seeds ───────────────────────────────────────────────────────────
import os, glob, random, time
import numpy as np, pandas as pd
import tensorflow as tf
from math import sqrt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense
from tensorflow.keras.optimizers import Adam

# reproducibility
seed = 42
np.random.seed(seed)
random.seed(seed)
tf.random.set_seed(seed)

# ─── PARAMETERS ────────────────────────────────────────────────────────────────
INPUT_DIR   = "data/raw"
OUTPUT_DIR  = "outputs"

TIMESTEPS   = 7    # sliding window length
DAY_PREDICT = 90   # days at end used as test set
EPOCHS      = 100
BATCH_SIZE  = 32

# ensure directories exist
os.makedirs(INPUT_DIR,  exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)


# ─── SHEET PROCESSOR ───────────────────────────────────────────────────────────
def process_sheet(df):
    # format dates & index
    df["Tarikh"] = pd.to_datetime(df["Tarikh"]).dt.strftime("%d/%m/%Y")
    df.index     = df["Tarikh"]
    df = df.drop("Tarikh", axis=1).apply(pd.to_numeric, errors="coerce")

    # extract series
    series = df["WholesalePriceNew"].values.reshape(-1, 1)

    # train/test split
    train_len = len(series) - DAY_PREDICT
    train, test = series[:train_len], series[train_len:]

    # scale to [0,1]
    scaler = MinMaxScaler((0, 1))
    train_s = scaler.fit_transform(train)
    test_s  = scaler.transform(test)

    # build training windows
    x_train, y_train = [], []
    for i in range(TIMESTEPS, len(train_s)):
        x_train.append(train_s[i-TIMESTEPS:i, 0])
        y_train.append(train_s[i, 0])
    x_train = np.array(x_train).reshape(-1, TIMESTEPS, 1)
    y_train = np.array(y_train)

    # build model
    model = Sequential([
        Conv1D(64, 3, activation="relu", input_shape=(TIMESTEPS, 1)),
        Flatten(),
        Dense(50, activation="relu"),
        Dense(1)
    ])
    model.compile(optimizer=Adam(1e-3), loss="mse")
    model.fit(x_train, y_train, epochs=EPOCHS,
              batch_size=BATCH_SIZE, verbose=2)

    # build test windows (include last train window)
    full = np.vstack((train_s[-TIMESTEPS:], test_s))
    x_test = [ full[i-TIMESTEPS:i, 0] for i in range(TIMESTEPS, len(full)) ]
    x_test = np.array(x_test).reshape(-1, TIMESTEPS, 1)

    # predict & unscale
    preds    = model.predict(x_test)
    preds_r  = scaler.inverse_transform(preds)
    y_test_r = test.reshape(-1, 1)

    # metrics
    rmse = sqrt(mean_squared_error(y_test_r, preds_r))
    mae  = mean_absolute_error(y_test_r, preds_r)
    mape = mean_absolute_percentage_error(y_test_r, preds_r) * 100

    return y_test_r.flatten(), preds_r.flatten(), rmse, mae, mape


# ─── MAIN LOOP ─────────────────────────────────────────────────────────────────
for fp in glob.glob(os.path.join(INPUT_DIR, "*_FillKNN.xlsx")):
    crop      = os.path.basename(fp).split("_")[0]
    out_fname = f"{crop}_CNN_results.xlsx"
    writer    = pd.ExcelWriter(os.path.join(OUTPUT_DIR, out_fname),
                               engine="xlsxwriter")

    print(f"\n➡️ Running CNN for {crop}…")
    xls = pd.ExcelFile(fp)
    for sheet in xls.sheet_names:
        df = pd.read_excel(fp, sheet_name=sheet)
        actual, preds, rmse, mae, mape = process_sheet(df)

        # assemble and save
        out_df = pd.DataFrame({
            "Actual":    actual,
            "Predicted": preds,
            "RMSE":      rmse,
            "MAE":       mae,
            "MAPE":      mape
        })
        out_df.to_excel(writer, sheet_name=sheet, index=False)
        print(f"   • {sheet}: RMSE={rmse:.3f}, MAE={mae:.3f}, MAPE={mape:.2f}%")

    writer.close()
    print(f"✅ Saved → {os.path.join(OUTPUT_DIR, out_fname)}")


'pip' is not recognized as an internal or external command,
operable program or batch file.


ModuleNotFoundError: No module named 'keras'